<a href="https://colab.research.google.com/github/mosigaaa/xlab-recommendation/blob/main/Recommendation_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/biodatlab/xlab-recommendation/blob/main/homework/homework.ipynb)

### 1. Implicit Recommendation with X-wines Dataset
Use [X-wines](https://github.com/rogerioxavier/X-Wines/tree/main/Dataset/last) dataset for implicit recommendation

##### Dataset preparation

In [1]:
# Download dataset from google drive
import gdown
output = "XWines_Slim_1K_wines_150K_ratings.zip"
url = "https://drive.google.com/file/d/1JuGLA56ibjPtzhupMxUHwA781XHT7Wml/view?usp=drive_link"
gdown.download(url=url, output=output, quiet=False, fuzzy=True)

# Extract dataset zip file
!unzip XWines_Slim_1K_wines_150K_ratings.zip

Downloading...
From: https://drive.google.com/uc?id=1JuGLA56ibjPtzhupMxUHwA781XHT7Wml
To: /content/XWines_Slim_1K_wines_150K_ratings.zip
100%|██████████| 2.47M/2.47M [00:00<00:00, 219MB/s]

Archive:  XWines_Slim_1K_wines_150K_ratings.zip
  inflating: XWines_Slim_150K_ratings.csv  
  inflating: XWines_Slim_1K_wines.csv  


In [ ]:
# TODO: Using implicit ALS model to get 10 wine recommendations for a single user.
# - Convert the dataset into an implicit one by setting ratings lower than 4.0 (or whatever threshold) to zero and those higher than 4.0 to one.
# - Weight the item-user sparse matrix with implicit BM25 weighting before creating an ALS model.
# - Display the result of the model with Gradio.

In [ ]:
!pip install implicit
!pip install gradio

In [3]:
import pandas as pd
import numpy as np

In [ ]:
Xwine_rating_1K = pd.read_csv("/content/XWines_Slim_1K_wines.csv")
Xwine_rating_150K = pd.read_csv("/content/XWines_Slim_150K_ratings.csv")
#print(Xwine_rating_1K.head(10))
print(Xwine_rating_150K.head(10))

In [5]:
Xwine_rating_150K.drop(columns=["Date"],inplace=True)

for index, rating in Xwine_rating_150K["Rating"].iteritems():
    if rating > 4:
        Xwine_rating_150K.at[index, "Rating"] = 1
    else:
       Xwine_rating_150K.at[index, "Rating"] = 0

<ipython-input-5-dffef22c8b4c>:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, rating in Xwine_rating_150K["Rating"].iteritems():


In [ ]:
print(Xwine_rating_150K["Rating"].value_counts())

In [7]:
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from implicit.nearest_neighbours import bm25_weight

In [10]:
Xwine_rating_150K["Rating"] = Xwine_rating_150K["Rating"].astype(float)
Xwine_rating_150K["UserID"] = Xwine_rating_150K["UserID"].astype("category")
Xwine_rating_150K["WineID"] = Xwine_rating_150K["WineID"].astype("category")

In [ ]:
X_rating = csr_matrix( (
    Xwine_rating_150K["Rating"],(Xwine_rating_150K["UserID"].cat.codes,Xwine_rating_150K["WineID"].cat.codes)
    )
)
X_rating = X_rating.tocsr()
print(X_rating)

In [ ]:
from implicit.nearest_neighbours import bm25_weight

X_rating = bm25_weight(X_rating, K1=100, B=0.8)
print(X_rating)
from implicit.als import AlternatingLeastSquares
model = AlternatingLeastSquares(factors=64, regularization=0.05, alpha=2.0)
# Implicit expect user-item (user-artist)
user_rating = X_rating.T.tocsr()

model.fit(user_rating)


In [21]:
import gradio as gr


def recommend_wine(user_id):
    user_id = int(user_id)
    if (
        Xwine_rating_150K["UserID"] == user_id
    ).any():  # check if input user_id exist in dataset or not
        selected_user_df = Xwine_rating_150K[Xwine_rating_150K["UserID"] == user_id].sort_values(
            by="Rating", ascending=False
        )

        # Get the user index for the user_id to use in the prediction
        user_idx = selected_user_df["UserID"].cat.codes.values[0]
        predicted_ratings = np.dot(W[user_idx], H)
        sort_rating_indices = np.flip(np.argsort(predicted_ratings))[:20]
        selected_predict_ratings = predicted_ratings[sort_rating_indices]

        # Get the WineID from the sorted indices
        rec_wine_ids = Xwine_rating_150K["WineID"].cat.categories[sort_rating_indices]

        rec_df = pd.DataFrame({
            "WineID": rec_wine_ids,
            "PredictedRating": selected_predict_ratings,
        })

        # Merge with the original dataset to get additional wine details
        rec_df = rec_df.merge(Xwine_rating_150K.drop_duplicates(subset=['WineID']), how="left", on="WineID")

        # Return the top 10 actual ratings by the user and top 10 recommendations
        return selected_user_df.head(10), rec_df.head(10)
    else:
        raise gr.Error("User ID not found")

# Gradio interface setup
demo = gr.Interface(
    fn=recommend_wine,
    inputs=gr.Textbox(label="Input UserID (e.g., from 1 - 610)"),
    outputs=[gr.DataFrame(label="User's Rated Wines"), gr.DataFrame(label="Recommended Wines")],
    examples=["21", "50"],
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4311dbb0f6ece6ebfd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### 2. Using CLIP to embed product images and recommend similar items
Use [fashion-product-images](https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset) dataset for CLIP image search

* Dataset: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset
* Interface with Kaggle API: https://www.kaggle.com/discussions/general/74235

##### Dataset preparation

In [ ]:
!pip install kaggle

In [31]:
# follow the "interface with Kaggle API" link, upload kaggle.json

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"makkawanchaimongkol","key":"c6687734fccb0256c9f2f480547ab078"}'}

In [32]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# test access kaggle dataset
!kaggle datasets list

In [34]:
# download dataset

!kaggle datasets download -d paramaggarwal/fashion-product-images-small

 99% 562M/565M [00:19<00:00, 33.2MB/s]
100% 565M/565M [00:19<00:00, 30.2MB/s]


In [ ]:
# extract dataset zip file

!unzip fashion-product-images-small.zipฤ

In [ ]:
! pip install torch ftfy regex tqdm numpy
! pip install openai-clip
! pip install gradio
! pip install gdown

In [38]:
import os
import os.path as op
from PIL import Image
from zipfile import ZipFile

import numpy as np
from tqdm import tqdm
import torch

import clip
# check available runtime

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
  ! pip install faiss-gpu
else:
  ! pip install faiss-cpu

print("Now running with " + device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00
Now running with cuda


In [39]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [41]:
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:12<00:00, 28.8MiB/s]


In [59]:
# load dataset

dataset_path = op.join(os.getcwd(), "/content/myntradataset/images/")
print(dataset_path)

/content/myntradataset/images/


In [60]:
# create list of all filename in dataset folder

all_folder_path = os.listdir(dataset_path)
all_folder_path.sort()
print(len(all_folder_path))

44441


In [87]:
# allocate embeddings array from num_file
embeddings_storage = np.zeros((len(all_folder_path),512),dtype=np.float32)

# encoder dataset & store images name
file_counter = 0
images_path = [f for f in os.listdir(dataset_path) if op.isfile(op.join(dataset_path, f))]
images_path.sort()
for path in tqdm(images_path):
    with torch.no_grad():
        image = (
            preprocess(Image.open(op.join(dataset_path, path))).unsqueeze(0).to(device)
        )
        embeddings_storage[file_counter] = np.array(
            model.encode_image(image).numpy(force=True)[0].astype("float32")
        )
        file_counter += 1



100%|██████████| 44441/44441 [12:00<00:00, 61.67it/s]


In [88]:
# create embeddings vector using FAISS

import faiss

index = faiss.IndexFlatL2(
    512
)  # dimension of 1 embedding decoded from CLIP model is 512
index.add(embeddings_storage)
print(index.ntotal)

44441


In [91]:
import gradio as gr
from PIL import Image
import numpy as np
import os.path as op
import torch



def recommend_similar_image(image_path):
    print(f"get image path {image_path}")

    test_image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

    with torch.no_grad():
        test_embeddings = model.encode_image(test_image).numpy(force = True)[0].astype("float32")
        test_embeddings = np.array([test_embeddings])

    k = 4  # number of recommendations
    square_distance, image_index = index.search(test_embeddings, k)
    print(image_index)
    print(square_distance)

    print("Opening Images...")
    recommended_images = [
        (
            Image.open(op.join(dataset_path, all_folder_path[image_index[0][i]])),
            f"Recommended Rank {i+1}",
        )
        for i in range(k)
    ]
    return recommended_images


demo = gr.Interface(
    fn=recommend_similar_image,
    inputs=gr.Image(type="filepath"),
    outputs=gr.Gallery(),
).launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4e8e1a3b5bf3ec3be4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


get image path /tmp/gradio/941c0fffed299a38aa131564a19d0d739f3ece66/image.png
[[    6 41468 15906 42396]]
[[ 0.       10.718597 11.229553 14.982812]]
Opening Images...
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://4e8e1a3b5bf3ec3be4.gradio.live


In [ ]:
def recommend_similar_image(text):

    text = clip.tokenize([text]).to(device)
    with torch.no_grad():
        test_embeddings = model.encode_text(text).numpy(force=True)[0].astype("float32")
        test_embeddings = np.array([test_embeddings])

    k = 10  # number of recommendations
    square_distance, image_index = index.search(test_embeddings, k)
    print(image_index)
    print(square_distance)

    print("Opening Images...")
    recommended_images = [
        (
            Image.open(op.join(dataset_path, all_folder_path[image_index[0][i]])),
            f"Recommended Rank {i+1}",
        )
        for i in range(k)
    ]
    return recommended_images


example_path = []
demo = gr.Interface(
    fn=recommend_similar_image,
    inputs=gr.Textbox(),
    outputs=gr.Gallery(),
).launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0d58e9a8da8d422b44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[[11286 33505 25976 12611 42482 13143  2598 20572 31812 17311]]
[[119.534355 120.381744 120.83953  122.36725  123.34304  123.40774
  123.564995 123.65398  123.70002  123.93276 ]]
Opening Images...
[[  661 28304 41571 38655 28299 16776 42585 42442 27336 28590]]
[[176.1723  179.19489 179.34119 179.75227 180.8226  181.25922 181.40344
  181.49213 181.76077 182.04488]]
Opening Images...
[[34227 28590 32743 42585  3796  4570 11286 28593 28895 11163]]
[[166.98631 167.86702 167.96751 167.99074 168.2092  168.29666 168.35619
  168.37315 168.41467 168.43793]]
Opening Images...
[[28081 43800 41571 28304 28299 37329 12611 28047 42442 11958]]
[[151.98413 152.98402 153.1293  154.42413 154.8538  154.895   155.10371
  155.32512 155.42017 155.5368 ]]
Opening Images...
[[20948 37351  9431 26266  4401  3125 30951 16603 38866  2516]]
[[127.45049 128.10587 128.65756 128.78375 129.0273  129.19585 129.55028
  129.66087 129.70586 129.72473]]
Opening Images...
[[28360 37329 28304 41571  3745 38866 41568 28299 2